###### Extracting features and use PNN to classify

In [1]:
import numpy as np
from scipy import stats
from wavelet_transform import bonn_dataset
raw_data = np.load("../Healthcare_signal_processing/Datasets/Bonn/data_all.npz")
decomposed_data = bonn_dataset()

[INFO] Dataset processing completed


###### Extract necessary features from time-domain raw signal

In [4]:
raw_data_healthy = np.vstack((raw_data['z'], raw_data['o']))   # healthy subjects
raw_data_inter_ictal = np.vstack((raw_data['n'], raw_data['f']))   # inter-ictal subjects
raw_data_ictal = np.vstack(raw_data['s'])                          # ictal subjects
raw_data_non_ictal = np.vstack((raw_data_healthy, raw_data_inter_ictal))  # non-ictal subjects
raw_data_all = np.vstack((raw_data_non_ictal, raw_data_ictal))   # all
raw_data_all.shape

(500, 4097)

###### Feature set 1: Statistical features

In [5]:
minn = np.amin(raw_data_all, axis = 1)
maxx = np.amax(raw_data_all, axis = 1)
rangee = maxx - minn
mean = np.mean(raw_data_all, axis = 1)
median = np.median(raw_data_all, axis = 1)
mode = np.array(stats.mode(raw_data_all, axis = 1)[0]).reshape(-1)
kurt = stats.kurtosis(raw_data_all, axis = 1)
skew = stats.skew(raw_data_all, axis = 1)
first_qua = np.quantile(raw_data_all, 0.25, axis = 1)
third_qua = np.quantile(raw_data_all, 0.75, axis = 1)
inter_qua = third_qua - first_qua
std = np.std(raw_data_all, axis = 1)
time_domain_stat_features = np.vstack((minn, maxx, rangee, mean, median, mode, kurt, skew, first_qua, 
                                 third_qua, inter_qua, std)).T
print("[INFO] Time domain statistical features extracted")

(500, 12)

###### Feature set 2: Hjorth based features

In [13]:
import pyeeg
hjorth_mob = []
hjorth_com = []
for samp in raw_data_all:
    hjorth = pyeeg.hjorth_mobility_complexity.hjorth(samp)
    hjorth_mob.append(hjorth[0])
    hjorth_com.append(hjorth[1])
hjorth_mob = np.array(hjorth_mob)
hjorth_com = np.array(hjorth_com)
hjorth_act = np.var(raw_data_all, axis = 1)
time_domain_hjorth_features = np.vstack((hjorth_mob, hjorth_com, hjorth_act)).T
print("[INFO] Time domain Hjorth features extracted")

[INFO] Time domain Hjorth features extracted


###### Feature set 3: Special time domain features

In [22]:
dfa = []
pfd = []
for samp in raw_data_all:
    #hurst.append(pyeeg.hurst(samp))
    dfa.append(pyeeg.detrended_fluctuation_analysis.dfa(samp))
    pfd.append(pyeeg.fractal_dimension.pfd(samp))
time_domain_special_features = np.vstack((np.array(dfa), np.array(pfd))).T
print("[INFO] Time domain Hjorth features extracted")
time_domain_special_features.shape

[INFO] Time domain Hjorth features extracted


(500, 2)

###### Feature set 4: Wavelet based features

In [24]:
from pyentrp import entropy
sample = []
shan = []
perm = []
muscale = []
for samp in raw_data_all:
    sample.append(entropy.sample_entropy(samp, 4, 0.2 * np.std(samp)))
    shan.append(entropy.shannon_entropy(samp))
    perm.append(entropy.permutation_entropy(samp))
    muscale.append(entropy.multiscale_entropy(samp, 2))
wavelet_features = np.vstack((np.array(dfa), np.array(pfd))).T
print("[INFO] Wavelet domain entropy features extracted")
wavelet_features.shape

/home/vicky/anaconda3/lib/python3.7/site-packages/pyentrp/entropy.py:178: RuntimeWarning: divide by zero encountered in log
  sampen =  - np.log(Ntemp[1:] / Ntemp[:-1])
/home/vicky/anaconda3/lib/python3.7/site-packages/pyentrp/entropy.py:178: RuntimeWarning: invalid value encountered in true_divide
  sampen =  - np.log(Ntemp[1:] / Ntemp[:-1])


KeyboardInterrupt: 

In [ ]:
features = np.hstack((time_domain_stat_features, time_domain_hjorth_features, time_domain_special_features,
                     wavelet_features))